In [9]:
%load_ext watermark
%watermark -a "Sushmit Roy" -u -d 

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Sushmit Roy 
last updated: 2018-04-22


In [14]:
import pymc3
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected=True)

In [13]:
# Code 2.1
ways = np.array([0, 3, 8, 9, 0])
ways/ways.sum()
# Code 2.2
stats.binom.pmf(6, n=9, p=0.5)

array([0.  , 0.15, 0.4 , 0.45, 0.  ])

In [19]:
def posterior_grid_approx(grid_points=5,success=6, tosses=9):
    """
    Grid approximations
    """
    # define grid
    p_grid = np.linspace(0, 1, grid_points)
    # define prior
    prior = np.repeat(1,grid_points)
    # compute likelihood at each value in grid
    likelihood = stats.binom.pmf(success,tosses, p_grid)
    # compute product of likelihood and prior
    unstd_posterior = likelihood * prior
    # standardization
    posterior = unstd_posterior/sum(unstd_posterior)
    return p_grid, posterior



In [60]:
points = 5
w, n = 6, 9
p_grid, posterior = posterior_grid_approx(points, w, n)

# Create a trace
trace1 = go.Scatter(
    x = p_grid,
    y = posterior,
    showlegend = False
    
)
points = 20
w, n = 6, 9
p_grid, posterior = posterior_grid_approx(points, w, n)
trace2 = go.Scatter(
    x = p_grid,
    y = posterior,
    name = 'success = {}\n tosses = {}'.format(w, n)
)


fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('5 Points',
                                                          '20 Points'),print_grid=False)


fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout']['xaxis1'].update(title='Probability of Water')
fig['layout']['xaxis2'].update(title='Probability of Water')
fig['layout']

fig['layout']['yaxis1'].update(title='Posterior Probability')
fig['layout']['yaxis2'].update(title='Posterior Probability')
fig['layout'].update(height=400)

fig['layout'].update(paper_bgcolor='rgb(233,233,233)')
fig['layout'].update(plot_bgcolor='rgb(200, 200, 200)')

iplot(fig)